# BACKGROUND

### Background Reading

ECC is an asymmetric cryptographic protocol that relies on a trap-door function. 

$y^2 = x^3 + aX +b$
Weierstrass equation 

We define an operator called "point addition". This operator takes two points on some curve and produce a third point on the curve. Taking the set of points on an elliptic curve, point addition defines an Abelian group operation. 

Scalar multiplication is repeated point addition of the same point. $Q = 2P = P+P$. It turns out it is a trap door function, it's hard to find $n$ such that $Q=np$ given $Q$ and $P$

Geometrically we can think of point addition as taking two points, $P,Q$ with their $x,y$ coordinates. We draw a straight line that passes through both points. We continue this line until we get third intersection with curve which is point $R$ we reflect it around $y$ to get $R' = R(x, -y)$. The result of addition is $P+Q = R'$ 

What if we want to add two of the same points together $P+P$? We can't draw a unique line through one point, but we can pick a unique line by calculating the tangent line to the curve at the point. Calculate the tangent line at the point P. Continue to do so until the line intersects with curve at point $R$. Reflect this point as before $P + P = R' = R(x, -y)$ 

What happens if there is no third intersection? Sometimes we will pick two points $P,Q$ and the line will not touch the curve again. In this case we say that the line intersects with point $O$ which is a single point located at the end of every vertical line at infinity. Now point addition for elliptic curve is defined in 2D space with additional point located at infinity. 

![img](https://cryptohack.org/static/img/ECClines.svg)

The point $O$ acts as the identity operator of the group: $P + O = P$ and $P + (-P) = O$

**Definition:** An elliptic curve E is the set of solutions to a Weierstrass equation $E: y^2 = x^3 + aX + b$ together with a point at infinity $O$. The constants $a,b$ must satisfy the relationship $4a^3 + 27b^2 \neq 0$ to ensure there are no singularities on the curve. 

Formally let E be an elliptic curve, point addition has these properties: 
1) $P + O = O + P = P$ 
2) $P + (-P) = O$ 
3) $(P + Q) + R = P + (Q + R)$ 
4) $P + Q = Q + P$ 

In ECC we study elliptic curves over a finite field $F_p$. This means we look at the curve modulo the charactertistic $p$ and an elliptic curve will no longer be a curve but a collection of points whose $x,y$ are integers in $F_p$

In [2]:
print('crypto{abelian}')

crypto{abelian}


# STARTER 

### Point Negation

In pictures they were real numbers, to apply in cryptographic settings, we study elliptic curves which have coordinates in a finite field $F_p$

We no longer think of the elliptic curves as geometric object, but rather a set of points defined by 
$E(F_p) = \{(x,y) : x,y \in F_p ; y^2  =x^3 + ax + b \} \cup O$

In [30]:
p = 9739 
y = 6936 
x = 8045 

print('crypto{'+str(x)+','+str(-y%p)+'}') 

crypto{8045,2803}


### Point Addition

Algorithm for calculating point addition of two points $P+Q$ 
1) If $P=O$, then $P+Q=Q$
2) Otherwise, if $Q=O$, then $P+Q = P$ 
3) Otherwise, write $P=(x_1, y_1)$ and $Q(x_2,y_2)$
4) If $x_1 = x_2$ and $y_1 = -y_2$, then $P+Q = O$ 
5) Otherwise: 

    5.1) if $P \neq Q: \lambda = \frac{(y_2 - y_1)}{x_2 - x_1}$ 
    
    5.2) if $P = Q: \lambda = \frac{3x_1^2 +a}{2y_1}$
6) $x_3 = \lambda^2 - x_1 - x_2, y_3 = \lambda(x_1-x_3) - y+1$ 
7) $P+Q=(x_3,y_3)$

In [36]:
from math import floor

O = 'Infinity'
def inv(a,p):
        return pow(a, p-2, p)

class Point: 
    def __init__(self, x, y, a, b, p): 
        self.x = x 
        self.y = y 
        self.a = a 
        self.b = b 
        self.p = p 
       

    def __eq__(self, other): 
        if isinstance(other, Point): 
            return self.x == other.x and self.y == other.y 
        return False
  
    def __add__(self, Q):
            if isinstance(Q, str) and Q == O: 
                 return self 
            elif isinstance(self, str) and self == O: 
                 return Q 
             
            x1 = self.x 
            y1 = self.y 
            x2 = Q.x 
            y2 = Q.y

         
            if x1 == x2 and y1 == -y2%p: 
                return O
           
            if self == Q: 
                lamb = (3*x1*x1 + self.a)*inv(2*y1 ,p) 
            else: 
                lamb = (y2-y1)*inv(x2-x1,p)     
            x3 = (lamb * lamb - x1 - x2)%p 
            y3 = (lamb*(x1 - x3) - y1)%p

            return Point(x3, y3, a, b, p)
    
    def __mul__(self, n): 
            Q = self
            R = O 

            while n>0: 
                if n%2 == 1: 
                    R = R+Q 
                Q = Q+Q 
                n = floor(n/2)
               

            return R
    
    def __radd__(self, other):
        return self.__add__(other)


In [50]:
a = 497 
b = 1768 
p = 9739 
    

X = Point(5274, 2841, a, b, p)
Y = Point(8669, 740, a, b, p)

assert(X+Y == Point(1024, 4440, a, b, p))
assert(X+X == Point(7284, 2107, a, b, p))

P = Point(493, 5564, a, b, p)
Q = Point(1539, 4742, a, b, p)
R = Point(4403, 5202, a, b, p)

res = P+P+Q+R
print('crypto{' + str(res.x) + ',' + str(res.y) +'}')

crypto{4215,2162}


### Point Multiplication 

Scalar multiplication of two points is defined by repeated addition 

Double and add algorithm: 
Input $n,P$
1) Set $Q = P$ and $R = O$
2) Loop while n>0

    2.1) If $n \equiv 1 \bmod 2$ set $R=R+Q$

    2.2) Set $Q=2Q$ and $n= \lfloor \frac{n}{2} \rfloor$

    2.3) If $n>0$, continue with loop at Step 2. 

3) Return the point $R$, which $nP$  

In [54]:
a = 497 
b = 1768 
p = 9739 

P = Point(2339, 2213, a, b, p)
n = 7863 
res = P*n 
print('crypto{' + str(res.x) + ',' + str(res.y) +'}')

crypto{9467,2742}


### Curves and Logs

The Elliptic Curve Discrete Logarithm Problem(ECDLP) is the problem of finding an integer $n$ s.t. $Q=nP$ 

The most efficient algorithm for this is $p^\frac{1}{2}$

Alice and Bob want a shared secret so they can start encrypting messages with some symmetric cryptographic protocol 

Alice and Bob agree on a curve $E$, a prime $p$ and a generator point $G$

In ECC it is important that the order of $G$ is prime. 

Alice generates a secret random integer $n_A$ and calculates $Q_A = n_AG$ 

Bob generates a secret random integer $n_B$ and calculates $Q_B = n_BG$ 

Alice sends Bob $Q_A$ and Bob sends alice $Q_B$. Alice then calculates $n_AQ_B$ and Bob calculates $n_BQ_A$. Due to associativity of scalar multiplication $S=n_BQ_A=n_AQ_B$ 

In [69]:
import hashlib 

a = 497 
b = 1768 
p = 9739

G = Point(1804, 5368, a, b, p)

Q_A = Point(815, 3190, a, b, p)
n_B = 1829 
s = (Q_A*n_B).x 
m = hashlib.sha1()

m.update(str(s).encode("utf-8"))

print("crypto{"+m.hexdigest()+"}")

crypto{80e5212754a824d3a4aed185ace4f9cac0f908bf}


### Efficient Exchange

We don't need to send both $x,y$ given $x$, there are only 2 possible values of $y$


In [17]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib


def is_pkcs7_padded(message):
    padding = message[-message[-1]:]
    return all(padding[i] == len(padding) for i in range(0, len(padding)))

def sqrt(a,p): 
    return -pow(a,(p+1)//4,p)%p , pow(a,(p+1)//4,p)


def decrypt_flag(shared_secret: int, iv: str, ciphertext: str):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    # Decrypt flag
    ciphertext = bytes.fromhex(ciphertext)
    iv = bytes.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)

    if is_pkcs7_padded(plaintext):
        return unpad(plaintext, 16).decode('ascii')
    else:
        return plaintext.decode('ascii')

a = 497  
b = 1768 
p = 9739 

x = 4726 
n = 6534  

y_2 = (pow(x,3,) + a*x + b) %p

shared_secret = (Point(x, sqrt(y_2, p)[1],a,b,p)*n).x 
iv = 'cd9da9f1c60925922377ea952afc212c'
ciphertext = 'febcbe3a3414a730b125931dccf912d2239f3e969c4334d95ed0ec86f6449ad8'

print(decrypt_flag(shared_secret, iv, ciphertext))

crypto{3ff1c1ent_k3y_3xch4ng3}


# PARAMETER CHOICE 

### Smooth Criminal

https://en.wikipedia.org/wiki/Pohlig–Hellman_algorithm#:~:text=In%20group%20theory%2C%20the%20Pohlig,order%20is%20a%20smooth%20integer.
We have smooth order of a curve that's why we can solve ECDLP with Pohlig Hellman

In [14]:
from Crypto.Cipher import AES
from Crypto.Util.number import inverse
from Crypto.Util.Padding import pad, unpad
from collections import namedtuple
from random import randint
import hashlib
import os

Point = namedtuple("Point", "x y")

O = 'Origin'


def check_point(P: tuple):
    if P == O:
        return True
    else:
        return (P.y**2 - (P.x**3 + a*P.x + b)) % p == 0 and 0 <= P.x < p and 0 <= P.y < p


def point_inverse(P: tuple):
    if P == O:
        return P
    return Point(P.x, -P.y % p)


def point_addition(P: tuple, Q: tuple):
    # based of algo. in ICM
    if P == O:
        return Q
    elif Q == O:
        return P
    elif Q == point_inverse(P):
        return O
    else:
        if P == Q:
            lam = (3*P.x**2 + a)*inverse(2*P.y, p)
            lam %= p
        else:
            lam = (Q.y - P.y) * inverse((Q.x - P.x), p)
            lam %= p
    Rx = (lam**2 - P.x - Q.x) % p
    Ry = (lam*(P.x - Rx) - P.y) % p
    R = Point(Rx, Ry)
    assert check_point(R)
    return R


def double_and_add(P: tuple, n: int):
    # based of algo. in ICM
    Q = P
    R = O
    while n > 0:
        if n % 2 == 1:
            R = point_addition(R, Q)
        Q = point_addition(Q, Q)
        n = n // 2
    assert check_point(R)
    return R


def gen_shared_secret(Q: tuple, n: int):
    # Bob's Public key, my secret int
    S = double_and_add(Q, n)
    return S.x



def decrypt_flag(shared_secret, c, iv): 
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
   
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(c)
    return unpad(plaintext, 16)



p = 310717010502520989590157367261876774703
a = 2
b = 3

n = 47836431801801373761601790722388100620 # from sage math 

b_x = 272640099140026426377756188075937988094
b_y = 51062462309521034358726608268084433317
B = Point(b_x, b_y)

shared_secret = gen_shared_secret(B, n)

encrypted_flag = {'iv': '07e2628b590095a5e332d397b8a59aa7', 
                  'encrypted_flag': '8220b7c47b36777a737f5ef9caa2814cf20c1c1ef496ec21a9b4833da24a008d0870d3ac3a6ad80065c138a2ed6136af'}

print(decrypt_flag(shared_secret, bytes.fromhex(encrypted_flag['encrypted_flag']), bytes.fromhex(encrypted_flag['iv'])).decode('utf-8')) 

crypto{n07_4ll_curv3s_4r3_s4f3_curv3s}


### Exceptional Curves

https://wstein.org/edu/2010/414/projects/novotney.pdf

https://crypto.stackexchange.com/questions/70454/why-smarts-attack-doesnt-work-on-this-ecdlp

https://web.archive.org/web/20160312120253/http://www.hpl.hp.com/techreports/97/HPL-97-128.pdf

In [60]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib

def decrypt_flag(secret, iv, c):
    sha1 = hashlib.sha1()
    sha1.update(str(secret).encode('ascii'))
    key = sha1.digest()[:16]

    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(c),16)


def SmartAttack(P,Q,p):
    E = P.curve()
    Eqp = EllipticCurve(Qp(p), [ ZZ(t)  for t in E.a_invariants() ])

    P_Qp = Eqp.lift_x(ZZ(P.xy()[0]))
    Q_Qp = Eqp.lift_x(ZZ(Q.xy()[0]))

    p_times_P = p*P_Qp
    p_times_Q = p*Q_Qp

    x_P,y_P = p_times_P.xy()
    x_Q,y_Q = p_times_Q.xy()

    phi_P = -(x_P/y_P)
    phi_Q = -(x_Q/y_Q)
    k = phi_Q/phi_P

    return ZZ(k)

g = (3034712809375537908102988750113382444008758539448972750581525810900634243392172703684905257490982543775233630011707375189041302436945106395617312498769005,
    4986645098582616415690074082237817624424333339074969364527548107042876175480894132576399611027847402879885574130125050842710052291870268101817275410204850)

pk =  (4748198372895404866752111766626421927481971519483471383813044005699388317650395315193922226704604937454742608233124831870493636003725200307683939875286865,
    2421873309002279841021791369884483308051497215798017509805302041102468310636822060707350789776065212606890489706597369526562336256272258544226688832663757)

enc = {'iv': '719700b2470525781cc844db1febd994', 'encrypted_flag': '335470f413c225b705db2e930b9d460d3947b3836059fb890b044e46cbb343f0'}
iv = bytes.fromhex(enc['iv'])
encrypted_flag = bytes.fromhex(enc['encrypted_flag'])

b_x = 0x7f0489e4efe6905f039476db54f9b6eac654c780342169155344abc5ac90167adc6b8dabacec643cbe420abffe9760cbc3e8a2b508d24779461c19b20e242a38
b_y = 0xdd04134e747354e5b9618d8cb3f60e03a74a709d4956641b234daa8a65d43df34e18d00a59c070801178d198e8905ef670118c15b0906d3a00a662d3a2736bf
   

p = 0xa15c4fb663a578d8b2496d3151a946119ee42695e18e13e90600192b1d0abdbb6f787f90c8d102ff88e284dd4526f5f6b6c980bf88f1d0490714b67e8a2a2b77
a = 0x5e009506fcc7eff573bc960d88638fe25e76a9b6c7caeea072a27dcd1fa46abb15b7b6210cf90caba982893ee2779669bac06e267013486b22ff3e24abae2d42
b = 0x2ce7d1ca4493b0977f088f6d30d9241f8048fdea112cc385b793bce953998caae680864a7d3aa437ea3ffd1441ca3fb352b0b710bb3f053e980e503be9a7fece


E = EllipticCurve(GF(p), [a, b])
P = E(pk[0],pk[1])
G = E(g[0], g[1])
Q = E(b_x, b_y)

k = SmartAttack(G,P,p)
ss = (k*Q).xy()[0]

print(decrypt_flag(ss, iv, encrypted_flag).decode())


crypto{H3ns3l_lift3d_my_fl4g!}


### Micro Transmissions

We can use Pollig-Hellman algorithm to solve DLP because of small $p$ 

In [10]:
import hashlib 
from Crypto.Cipher import AES 
from Crypto.Util.Padding import unpad

flag =  {'iv': 'ceb34a8c174d77136455971f08641cc5', 'encrypted_flag': 'b503bf04df71cfbd3f464aec2083e9b79c825803a4d4a43697889ad29eb75453'}

ss = 71305760181590994283497011485670689323451437179425419742468846237902256821556
sha1 = hashlib.sha1() 
sha1.update(str(ss).encode('ascii'))
key = sha1.digest()[:16]

iv = bytes.fromhex(flag['iv'])
c = bytes.fromhex(flag['encrypted_flag'])

cipher = AES.new(key, AES.MODE_CBC, iv)
cipher.decrypt(c)

b'%\xc1\x870\xf1\x1bqVD{`\x9e)\x00Q\xec\xb4\x1b\xc2\xceG\x0fl\xe4\x11(A\x9f]D\xff\xb6'

### Elliptic Nodes

Resources: 
1. https://crypto.stackexchange.com/questions/97811/find-elliptic-curve-parameters-a-and-b-given-two-points-on-the-curve 
2. https://crypto.stackexchange.com/questions/61302/how-to-solve-this-ecdlp

First we have to find $a,b$ parameters for the curve [1]

Given two points $P=(x_1, y_1)$ $Q = (x_2, y_2)$ 

$y_1^2 = x_1^3 + ax_1 + b (\bmod N)$ 

$y_2^2 = x_2^3 + ax_2 + b (\bmod N)$

<hr>

$y_1^2 - y_2^2 = x_1^3 - x_2^3 + a(x_1 - x_2) (\bmod N)$ 

$a = (y_1^2 - y_2^2 - (x_1^3 - x_2^3)) * (x_1 - x_2)^{-1} (\bmod N)$


When we use sage to define an Elliptic Curve this way we get error that this is a singular curve, googling how to solve DLP on singular curve leads to [2]

Singular point is when $\frac{dy}{dx} = \frac{dx}{dy} = \frac{0}{0}$ 

We translate curve to have singular point at $(0,0)$ by changing variables 
$(x,y) -> (x-s, y-0)$

This curve is a node, we can map it to multiplicative group where we solve DLP easy. Because our singular point coordinate has square modulo $p$, we know it maps to $F^*_p$ where we solve DLP


In [4]:
from Crypto.Util.number import long_to_bytes
print(long_to_bytes(175707932493016342199601625200584496546434097133638117913010244817446203005).decode()) 

crypto{s1ngul4r_s1mplif1c4t1on}


### Moving Problems

Resources:
1. https://crypto.stackexchange.com/questions/1871/how-does-the-mov-attack-work
2. https://crypto.stanford.edu/pbc/notes/elliptic/movattack.html
3. https://math.stackexchange.com/questions/824123/what-is-an-embedding-degree-of-elliptic-curve

The MOV attack uses a bilinear pairing, which is a function $e$ that maps two points in an Elliptic curve $E(\mathbb F_q)$ to an element in the finite field $F_q^k$ where $k$ is the embedding degree for the curve. 

The pairing is considered to be secure if taking discrete log both in $E(\mathbb F_q)$ and $\mathbb F^{*}_{q^k}$ are computationally infeasible 

The bilinearity means that $e(rP, sQ) = e(P,Q)^{rs}$ for points $P,Q$. Therefore if we want to compute the discrete logarithm for $rP$, we can instead compute $u=e(P,Q)$ and $v = e(rP,Q)$ for any $Q$. Due to bilinearity we have that $v=e(P,Q)^r = u^r$. Now we can solve discrete log in $F_{q^k}$.

We calculate $k$ with this theorem : 

Let $E$ be an elliptic curve defined over a finite field $\mathbb F_q$, let $n$ be a prime dividing $\#E(\mathbb F_q)$. The embedding degree of $E$ with respect to $n$ is the smallest integer $k$ s.t. $n$ divides $q^k-1$ 

So we find the number of points on elliptic curve ($\#E(\mathbb F_q$) and find $k$

In [2]:
import hashlib 
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

G = (479691812266187139164535778017 , 568535594075310466177352868412)
Pa =  (1110072782478160369250829345256 , 800079550745409318906383650948)
Pb =  (1290982289093010194550717223760 , 762857612860564354370535420319)
encrypted_flag =  {'iv': 'eac58c26203c04f68d63dc2c58d79aca', 
                   'encrypted_flag': 'bb9ecbd3662d0671fd222ccb07e27b5500f304e3621a6f8e9c815bc8e4e6ee6ebc718ce9ca115cb4e41acb90dbcabb0d'}

ss = 57514367079882430785803122958


iv = bytes.fromhex(encrypted_flag['iv'])
c = bytes.fromhex(encrypted_flag['encrypted_flag'])


sha1 = hashlib.sha1()
sha1.update(str(ss).encode('ascii'))
key = sha1.digest()[:16]
   
cipher = AES.new(key, AES.MODE_CBC, iv)
plaintext = cipher.decrypt(c)
unpad(plaintext, 16)


b'crypto{MOV_attack_on_non_supersingular_curves}'

# PARAMETER CHOICE 2

### Real Curve Crypto

https://hackmd.io/@grhkm/By-_iF795

https://math.mit.edu/classes/18.783/2015/LectureNotes15.pdf

In [1]:
d = {"gx": "1.15939524880832589559531697886995971202850341796875", 
 "gy": "0.63171256444643032392780992695525219049591162225190317013257297329770648576821328127894482837966831464341245854394869021441094534520252153121771214144624262228503025100001998247467928012492847829951495", 
 "px": "1052.1869486109503324827555468817188804055933729601321435932864694301534931492427433020783168479195188024409373571681097603398390379320742186401833284576176214641603772370675124838606986281131453644941", 
 "py": "34130.226434169760878074808301335090475271836563983186858174748793301185920507814323732035620490086239117019147179225251544229354849757884152095399018744815803126126630119173317092410331587882005521149", 
 "ciphertext": "a104b68d30a207eabf293324fbde64f8d628fb07068058c1e76e670e7e805fc567f739185bbe6cbb44f09013173ee653", 
 "iv": "485f9a1e4a3b19348367280df13f9e77"}

# SIGNATURES

### Digestive

On wikipedia of ECDSA it states that: 
- We use the $L_n$ leftmost bits of $e$ as $z$ and use that in other calculations 

Since it uses nist 192 curve, it is actually 192 bits that are used, or 24 bytes which means that username or whatever is after doesn't affect verification, so we can just send admin:true in our json and be done 

In [28]:
import requests
import json 
url = 'https://web.cryptohack.org/digestive/'

data = json.loads(requests.get(url+"sign/" + 'aaa').text)
sig = data['signature']

msg = '{"admin": false, "username": "aaa", "admin":true}'
print(json.loads(requests.get(url +"verify/" + msg +"/" + sig).text)['flag'])

crypto{thanx_for_ctf_inspiration_https://mastodon.social/@filippo/109360453402691894}


### Curveball

Because generator is not checked we can abuse this fact and send our pk with forged private key d, so basically we could use one to get Q*1 = Q 

But since check is implemented we send whatever number we want for private key and send inverse of this number multiplied by Q, so they cancel out and we are left with public key, being verified as the wrong entity

In [21]:
import telnetlib
import json
import fastecdsa
from fastecdsa.point import Point
from Crypto.Util.number import inverse

HOST = "socket.cryptohack.org"
PORT = 13382

G = fastecdsa.curve.P256.G
assert G.x, G.y == [0x6B17D1F2E12C4247F8BCE6E563A440F277037D812DEB33A0F4A13945D898C296,
                    0x4FE342E2FE1A7F9B8EE7EB4A7C0F9E162BCE33576B315ECECBB6406837BF51F5]


tn = telnetlib.Telnet(HOST, PORT)

def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline()
    return json.loads(line.decode())

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)

print(readline())
pk =  Point(0x3B827FF5E8EA151E6E51F8D0ABF08D90F571914A595891F9998A5BD49DFA3531, 0xAB61705C502CA0F7AA127DEC096B2BBDC9BD3B4281808B3740C320810888592A)
              
gen = inverse(3, fastecdsa.curve.P256.q)*pk 

msg =   {"host":"www.bing.com",
         "private_key": 3, 
         "curve": "secp256r1",
         "generator": [gen.x, gen.y]}

json_send(msg)

print(readline())


b'Welcome to my secure search engine backed by trusted certificate library!\n'
b'"Hey bing! Tell me about crypto{Curveballing_Microsoft_CVE-2020-0601}"\n'


### ProSign 3

This is basically PS3 nonce reuse attack on ECDSA, because of 

```python
randrange(1, n)
```

Where n is seconds, so we will send attack query for 2 because only possible value is 1, and for 3 as well so we can get either 1,2 so it will work 1/2 the times, so expected would be 2 runs. 

Now we derive formula for reused nonce to get the private key, we sign arbitrary message then 

$s_1 = k^{-1}(H(m_1) + r d)$

$s_2 = k^{-1}(H(m_2) + r d)$

From here we can calculate $k$ 

$s_1k = H(m_1) + rd $ 

$s_2k = H(m_2) + rd $ 

$k(s_2 - s_1) = H(m_2) - H(m_1)$ 

$k = (s_2 - s_1)^{-1} (H(m_2) - H(m_1))$

When we plug this in 

$s_2k = H(m_2) + rd$ 

$s_2k - H(m_2) = rd$ 

$(s_2k - H(m_2))*r^{-1} = d$ 


In [31]:
import telnetlib
import json
import hashlib 
from datetime import datetime 
from ecdsa.ecdsa import Public_key, Private_key, Signature, generator_192
import time 

from Crypto.Util.number import bytes_to_long

HOST = "socket.cryptohack.org"
PORT = 13381

g = generator_192
n = g.order()

tn = telnetlib.Telnet(HOST, PORT)

def readline():
    return tn.read_until(b"\n")

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)

def sha1(m): 
    sha1_hash = hashlib.sha1()
    sha1_hash.update(m)
    return sha1_hash.digest()

def send(t): 
    while True: 
        now = datetime.now()
        if now.strftime("%S") == t:
            m = dict()
            m['option'] = 'sign_time' 
            json_send(m)
            return json.loads(readline().decode('utf-8')) 
        time.sleep(1) 

print(readline())

response = send('02')
s1, r1, h1 = int(response['s'],16), int(response['r'],16), bytes_to_long(sha1(response['msg'].encode())) 


response = send('03') 
s2, r2, h2 = int(response['s'],16), int(response['r'],16), bytes_to_long(sha1(response['msg'].encode())) 

r = r1 = r2 
print(r1 == r2)

k = pow((s2 - s1), -1, n) * (h2 - h1)%n 
d = pow(r, -1, n) * ((s1*k) - h1)%n 

pk1, _= Signature(r1, s1).recover_public_keys(h1, g)
sk1 = Private_key(pk1, d) 

forgery = dict() 
forgery['msg'] = 'unlock'
forgery['option'] = 'verify'

h = bytes_to_long(sha1("unlock".encode()))

sig = sk1.sign(h, 1) 
forgery['r'] = hex(sig.r) 
forgery['s'] = hex(sig.s) 

json_send(forgery)
print(json.loads(readline().decode('utf-8'))['flag'])



b'Welcome to ProSign 3. You can sign_time or verify.\n'
True
crypto{ECDSA_700_345y_70_5cr3wup}


### No Random, No Bias

https://eprint.iacr.org/2019/023.pdf

https://blog.trailofbits.com/2020/06/11/ecdsa-handle-with-care/

# EDWARDS CURVES

### Edwards Goes Degenerate

Name kinda suggested it you can find it explained here: 
https://eprint.iacr.org/2015/1233.pdf 

We reduce the problem to solving discrete log in the multiplicative group of the base field. 

Theorem: Let $E_{a,d}$ be a twisted Edwards curve over $F_p$. The subset $\widetilde{G} \subset \mathbb{F}_p^2$ of the affine plane consisting of points of the form $(0,y),y \neq 0$ endowed with the addition law defined by the same formula as $E_{a,d}$ given by equation, forms a group isomorphic to $\mathbb{F}^*_p$ under the isomorphism $y \mapsto (0,y)$

We chose the invalid point $\widetilde{P}$ of the form $(0, \widetilde{y})$ and recieve as a result the scalar multiplication of $\widetilde{P}$ by the secret $k$ in the group $\widetilde{G}$ or $(\widetilde{y^k})$, now we get $k$ by solving discrete log in $F^*_p$ 

In [8]:
from Crypto.Util.number import inverse, bytes_to_long
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from random import randint
from hashlib import sha1
import os

FLAG = b'crypto{????????????????????????????????????}'


class TwistedEdwards():
    # Elliptic curve in Edwards form:
    # -x**2 + y**2 = 1 + d*x**2*y**2
    # birationally equivalent to the Montgomery curve:
    # y**2 = x**3 + 2*(1-d)/(1+d)*x**2 + x

    def __init__(self, p, d, order, x0bit, y0):
        self.p = p
        self.d = d
        self.order = order
        self.base_point = (x0bit, y0)

    def recover_x(self, xbit, y):
        xsqr = (y**2 - 1)*inverse(1 + self.d*y**2, self.p) % self.p
        x = pow(xsqr, (self.p + 1)//4, self.p)
        if x**2 == xsqr :
            if x & 1 != xbit:
                return p - x
            return x
        return 0

    def decompress(self, compressed_point):
        xbit, y = compressed_point
        x = self.recover_x(xbit, y)
        return (x, y)

    # complete point addition formulas
    def add(self, P1, P2):
        x1, y1 = P1
        x2, y2 = P2
        
        C = x1*x2 % self.p
        D = y1*y2 % self.p
        E = self.d*C*D
        x3 = (1 - E)*((x1 + y1)*(x2 + y2) - C - D) % self.p
        y3 = (1 + E)*(D + C) % self.p
        z3 = 1 - E**2 % self.p
        z3inv = inverse(z3, self.p)
        return (x3*z3inv % self.p, y3*z3inv % self.p)

    # left-to-right double-and-add
    def single_mul(self, n, compressed_point):
        P = self.decompress(compressed_point)        
        t = n.bit_length()
        if n == 0:
            return (0,1)
        R = P
        for i in range(t-2,-1,-1):
            bit = (n >> i) & 1
            R = self.add(R, R)
            if bit == 1:
                R = self.add(R, P)
        return (R[0] & 1, R[1])


def gen_key_pair(curve):
    n = randint(1, curve.order-1)
    P = curve.single_mul(n, curve.base_point)
    return n, P
    
def gen_shared_secret(curve, n, P):
    xbit, y = curve.single_mul(n, P)
    return y
    

def encrypt_flag(shared_secret: int):
    # Derive AES key from shared secret
    key = sha1(str(shared_secret).encode('ascii')).digest()[:16]
    # Encrypt flag
    iv = os.urandom(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(FLAG, 16))
    # Prepare data to send
    data = {}
    data['iv'] = iv.hex()
    data['encrypted_flag'] = ciphertext.hex()
    return data




p = 110791754886372871786646216601736686131457908663834453133932404548926481065303
order = 27697938721593217946661554150434171532902064063497989437820057596877054011573
d = 14053231445764110580607042223819107680391416143200240368020924470807783733946
x0bit = 1
y0 = 11
curve = TwistedEdwards(p, d, order, x0bit, y0)

pA =  (0, 109790246752332785586117900442206937983841168568097606235725839233151034058387)
pB =  (0, 45290526009220141417047094490842138744068991614521518736097631206718264930032)

encrypted_flag =  {'iv': '31068e75b880bece9686243fa4dc67d0', 
                   'encrypted_flag': 'e2ef82f2cde7d44e9f9810b34acc885891dad8118c1d9a07801639be0629b186dc8a192529703b2c947c20c4fe5ff2c8'}

a = 22177185339821817642584340290303072361216253354374422848549320419774574392697 # from sagemath code 
shared_secret = gen_shared_secret(curve, a, pB) 

key = sha1(str(shared_secret).encode('ascii')).digest()[:16]
iv = bytes.fromhex(encrypted_flag['iv']) 
c = bytes.fromhex(encrypted_flag['encrypted_flag'])

cipher = AES.new(key, AES.MODE_CBC, iv)
print(unpad(cipher.decrypt(c),16).decode('utf-8')) 

crypto{degenerates_will_never_keep_a_secret}


# SIDE CHANNELS 

### Montgomery's Ladder

Input: $P \in E(F_p)$ and $l$-bit integer $k = \Sigma 2^i k_i$ where $k_{i-1} = 1$

Output: $[k]P \in E(F_p)$

1. Set ($R_{\empty}, R_1$) to ($P,[2]P$)
2. for i = l-2 down to $\empty$ do 
3. If $k_i = \empty$ then  
4. Set($R_{\empty}, R_1$) to ($[2]R_{\empty}, R_{\empty} +R_1$)  
5. Else: 
6. Set($R_{\empty}, R_1$) to ($R_{\empty} +R_1, [2]R_{1}$)
7. Return $R_{\empty}$

Montogomery curves: $By^2 = x^3 + Ax^2 + x$ 

In [49]:
def add_montgomery(P,Q): 
    x1 = P.x 
    y1 = P.y 
    x2 = Q.x 
    y2 = Q.y 
    p = P.p 

    alpha = (y2-y1)*inv(x2-x1,p)  
    x3 = P.b * alpha * alpha - P.a - x1 - x2 
    y3 = alpha * (x1-x3) - y1 

    return Point(x3%p, y3%p, P.a, P.b, P.p)

def double_montgomery(P): 
    x1 = P.x 
    y1 = P.y 
    p = P.p 

    alpha = (3*x1*x1 + 2 * P.a * x1 + 1)*inv(2*P.b*y1,p) 
    x3 = P.b*alpha*alpha - P.a - 2*x1 
    y3 = alpha * (x1 - x3) - y1 

    return Point(x3%p, y3%p, P.a, P.b, P.p)

def mul_montgomery(k,P): 
    l = len(bin(k)) - 2 # because of 0b 
    R0 = P 
    R1 = double_montgomery(P) 

    for i in range(l-2, -1, -1): 
        ki = (k >> i) & 1 
        if ki == 0: 
            R0, R1 = double_montgomery(R0), add_montgomery(R0, R1) 
        else: 
            R0, R1 = add_montgomery(R0, R1), double_montgomery(R1) 

    return R0 

def legendre(a, p):
    return pow(a, (p - 1) // 2, p)

def tonelli(n, p):
    assert legendre(n, p) == 1, "not a square (mod p)"
    q = p - 1
    s = 0
    while q % 2 == 0:
        q //= 2
        s += 1
    if s == 1:
        return pow(n, (p + 1) // 4, p)
    for z in range(2, p):
        if p - 1 == legendre(z, p):
            break
    c = pow(z, q, p)
    r = pow(n, (q + 1) // 2, p)
    t = pow(n, q, p)
    m = s
    t2 = 0
    while (t - 1) % p != 0:
        t2 = (t * t) % p
        for i in range(1, m):
            if (t2 - 1) % p == 0:
                break
            t2 = (t2 * t2) % p
        b = pow(c, 1 << (m - i - 1), p)
        r = (r * b) % p
        c = (b * b) % p
        t = (t * c) % p
        m = i
    return r


p = pow(2,255) - 19
B = 1 
a = 486662 
E = lambda x,p: (x**3 + a*x*x + x) %p 
k = int('0x1337c0decafe',16)
G = Point(9, tonelli(E(9,p),p), a, b, p) 

print('crypto{'+str(mul_montgomery(k, G).x)+'}')  
 

crypto{49231350462786016064336756977412654793383964726771892982507420921563002378152}


### Double and Broken 

In [15]:
import ast 
import numpy as np 
from Crypto.Util.number import long_to_bytes
import matplotlib.pyplot as plt 


with open('data/collected_data.txt', 'r') as file:
    data = file.read()

power_list = np.array(ast.literal_eval(data))
avg = np.mean(power_list)

bits = (np.mean(power_list, axis = 0)>avg).astype(int)
bits = bits[::-1]

binary_string = ''.join(str(bit) for bit in bits)
l = int(binary_string,2)

print(long_to_bytes(int(l)).decode('utf-8'))

crypto{Sid3_ch4nn3ls_c4n_br34k_s3cur3_curv3s}
